# Scrape Data from Shopee with beautifulsoup and selenium
### Data for analysing total amount discounted price from the buyer, which will include:
- ใช้โค้ด Shopee แล้ว
- ใช้โค้ดส่วนลดของร้านค้า
- ส่วนลดค่าจัดส่ง
- ส่วนลด Shopee Coins
- รวมการสั่งซื้อ
- รวมค่าสินค้า
หากผิดพลาดประการใดแจ้งได้เลยครับ

In [94]:
# !pip install selenium
# !pip install bs4
# !pip install python-dotenv

DEPRECATION: textract 1.6.5 has a non-standard dependency specifier extract-msg<=0.29.*. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of textract or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2.1 -> 23.3
[notice] To update, run: C:\Users\DeRoxy\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


## Code ที่ต้อง Import

In [50]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys
from dotenv import dotenv_values
import bs4
import time
import re

## Experiment - Play Ground

### Selenium เพื่อการเข้า Web

In [51]:
#Open browser ด้วย Selenium
options = Options()
options.binary_location = r'C:\Program Files\Mozilla Firefox\firefox.exe'
driver = webdriver.Chrome()
driver.get('https://shopee.co.th/')


In [53]:
#เลือกภาษาไทย Selenium
thai_button = driver.find_element('xpath','/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
thai_button.click()
#time.sleep(3)

In [54]:
#ปิดโฆษณา Selenium
close_adver = driver.execute_script('return document.querySelector("shopee-banner-popup-stateful").shadowRoot.querySelector("div.shopee-popup__close-btn")')
close_adver.click()

In [55]:
#Search ใน Search bar ของ Shopee
search = driver.find_element('xpath','//*[@id="main"]/div/header/div[2]/div/div[1]/form/div/div[1]/input')
search.send_keys('computer')

In [56]:
#กด Enter
search.send_keys(Keys.ENTER)

In [57]:
#login
from dotenv import dotenv_values

# Load the environment variables from the .env file
env_vars = dotenv_values('.env')

# Access the values using the keys
username = env_vars['USERNAME']
password = env_vars['PASSWORD']
# Locate the username input field and enter the username
username_field = driver.find_element("name", "loginKey")
username_field.clear()  # Clear any existing text in the field
username_field.send_keys(username)
time.sleep(0.5)

# Locate the password input field and enter the password
password_field = driver.find_element("name", "password")
password_field.clear()  # Clear any existing text in the field
password_field.send_keys(password)
time.sleep(0.5)

password_field.send_keys(Keys.ENTER)


In [58]:
# test_url = 'https://shopee.co.th/user/purchase/order/150628041206898?type=6'
test_url = 'https://shopee.co.th/user/purchase/order/150628007234193?type=6'
test_url = 'https://shopee.co.th/user/purchase/order/128841462252171?type=7'

driver.get(test_url)
time.sleep(1)

### Scraping Process

In [59]:
data = driver.page_source #ดึงข้อมูลจากหน้าเว็บ
soup = bs4.BeautifulSoup(data) #จัดในรูปแบบ BeautifulSoup

In [60]:
elements = "#main > div > div.wf9gag > div > div.xMDeox > div > div > main > section:nth-child(12) > div > section > div.RZJjTX > div"
soup.select(elements) #css selector

[<div class="TokOv1"><div class="_8kMYJ3"><span>รวมค่าสินค้า</span></div><div class="CxyZBG"><div>฿2,190</div></div></div>,
 <div class="TokOv1"><div class="_8kMYJ3"><span>ค่าจัดส่ง</span></div><div class="CxyZBG"><div>฿134</div></div></div>,
 <div class="TokOv1"><div class="_8kMYJ3"><span>ใช้ 144 Shopee Coins</span></div><div class="CxyZBG"><div>-฿144</div></div></div>,
 <div class="TokOv1 a59vwO"><div class="_8kMYJ3 B6pCRN"><span>รวมการสั่งซื้อ</span></div><div class="CxyZBG"><div class="_8ZGgbl">฿2,180</div></div></div>]

In [61]:
el=soup.select(elements)
len(el)

4

### Scrape

In [62]:
#ดึงข้อมูลชื่อ
for e in el:
    title = e.select_one(f" div.TokOv1 > div._8kMYJ3 > span").text.strip()
    price = e.select_one(f" div.TokOv1 > div.CxyZBG > div").text.strip().replace("฿", "")
    print(test_url, title, price)

https://shopee.co.th/user/purchase/order/128841462252171?type=7 รวมค่าสินค้า 2,190
https://shopee.co.th/user/purchase/order/128841462252171?type=7 ค่าจัดส่ง 134
https://shopee.co.th/user/purchase/order/128841462252171?type=7 ใช้ 144 Shopee Coins -144
https://shopee.co.th/user/purchase/order/128841462252171?type=7 รวมการสั่งซื้อ 2,180


## Scrape ข้อมูลทั้งหมดที่ต้องการ

### Login function

In [65]:
# Function to perform login using Selenium (if needed)
def perform_login(username, password):
    # Configure the webdriver (make sure to specify the path to your webdriver)
    # driver = webdriver.Edge('C:\\ProgramData\\Microsoft\\Windows\\Start Menu\\Programs\\Microsoft Edge.lnk')
    driver = webdriver.Chrome()

    # Navigate to the login page
    driver.get('https://shopee.co.th/buyer/login')
    time.sleep(2)  # Add a delay to ensure page loading

    # Handle the language selection popup dialog
    # Find the button element by class name and text
    thai_button = driver.find_element('xpath','/html/body/div[2]/div[1]/div[1]/div/div[3]/div[1]/button')
    thai_button.click()

    # Locate the username input field and enter the username
    username_field = driver.find_element("name", "loginKey")
    username_field.clear()  # Clear any existing text in the field
    username_field.send_keys(username)
    time.sleep(0.5)

    # Locate the password input field and enter the password
    password_field = driver.find_element("name", "password")
    password_field.clear()  # Clear any existing text in the field
    password_field.send_keys(password)
    time.sleep(0.5)

    password_field.send_keys(Keys.ENTER)
    time.sleep(1.5)

    return driver


### Scraping function

In [73]:
def scrape_data(url, driver):

    # Navigate to the URL
    driver.get(url)
    time.sleep(2)

    # Get the data
    data = driver.page_source
    soup = bs4.BeautifulSoup(data)

    # Extract the Elements
    elements = "#main > div > div.wf9gag > div > div.xMDeox > div > div > main > section:nth-child(12) > div > section > div.RZJjTX > div"
    el = soup.select(elements)

    data = []
    # Extract data in elements
    for e in el:
        title = e.select_one(f" div.TokOv1 > div._8kMYJ3 > span").text.strip()
        price = e.select_one(f" div.TokOv1 > div.CxyZBG > div").text.strip().replace("฿","").replace(",", "").replace(" ", "")
        price = float(price)
        extracted = [url, title, price]
        data.append(extracted)

    return data

### Main()

In [77]:
# Load the environment variables from the .env file
env_vars = dotenv_values('.env')
username = env_vars['USERNAME']
password = env_vars['PASSWORD']

with open('purchased-link.txt', 'r') as f:
    urls = f.readlines()

records = []
nlines = len(urls)

driver = perform_login(username, password)
for i in range(nlines):
    url = urls[i].strip().split("?")[0]
    print(f"{i+1}/{nlines}", url)

    data = scrape_data(url, driver)
    for d in data:
        records.append(d)
    time.sleep(0.5)

df = pd.DataFrame(records, columns=['url', 'title', 'price'])
df
df.to_csv('Shopee_Purchased.csv', encoding='utf-8', index=False)
driver.quit()

1/207 https://shopee.co.th/user/purchase/order/127121523237033
2/207 https://shopee.co.th/user/purchase/order/127121523237034
3/207 https://shopee.co.th/user/purchase/order/127235762277872
4/207 https://shopee.co.th/user/purchase/order/127121523237032
5/207 https://shopee.co.th/user/purchase/order/127816839246030
6/207 https://shopee.co.th/user/purchase/order/127235762277868
7/207 https://shopee.co.th/user/purchase/order/127235762277873
8/207 https://shopee.co.th/user/purchase/order/127708734208715
9/207 https://shopee.co.th/user/purchase/order/127708734208716
10/207 https://shopee.co.th/user/purchase/order/127235762277869
11/207 https://shopee.co.th/user/purchase/order/128841462252171
12/207 https://shopee.co.th/user/purchase/order/127708734208718
13/207 https://shopee.co.th/user/purchase/order/127708734208717
14/207 https://shopee.co.th/user/purchase/order/127811341287858
15/207 https://shopee.co.th/user/purchase/order/128841462252172
16/207 https://shopee.co.th/user/purchase/order/1